# Multivariate Time Series Analysis Notebook

## 1.1 Install the Necessary Packages

In [ ]:
!pip install ibm_db

#### Restart the Kernel `Kernel>Restart`

## 1.2 Import the Packages

In [ ]:
from sklearn import preprocessing
from matplotlib import pyplot as plt
import numpy as np

import ibm_db

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf, pacf

## 2. Configuration

### 2.1 Insert the Db Credentials

Ensure to rename as `credentials_1`

In [ ]:
# The following code contains the credentials for a connection in your Project.
# You might want to remove those credentials before you share your notebook.

from project_lib import Project
project = Project.access()
credentials_1 = project.get_connection(name="Smruthi Db2 on Cloud")

### 2.2 Insert the Dataset

Click on the `10/01` icon and then click on `Insert Pandas Dataframe`. Ensure the name is `df_data_1`

## 3. Pre-Processing

### 3.1 Prepare dataset for Time-Series Model

In [ ]:
# Make column datatype a datetime object
df_data_1['Start_Time_MM_DD_YYYY'] = pd.to_datetime(df_data_1.Start_Time_MM_DD_YYYY , format = '%Y%m%d')




In [ ]:
df_data_1

In [ ]:
# Set the date column as index for dataset
data = df_data_1.drop(['Start_Time_MM_DD_YYYY'], axis=1)
data.index = df_data_1.Start_Time_MM_DD_YYYY

In [ ]:
''' NOTE: This example is given so that, ensure there are no nan's in your dataset 
    and they are replaced with type consistent missing values

data[' _dewptm']=data[' _dewptm'].fillna(1000)
'''

## 4. Time-Series Model Training

### 4.1 Train your AR Model

In [ ]:
ts_data=data['Call Dropped']
ts_data

In [ ]:
ts_log = np.log(ts_data)

In [ ]:
ts_log_diff = ts_log - ts_log.shift()

In [ ]:
ts_log_diff.dropna(inplace=True)
ts_log_diff

In [ ]:
model = ARIMA(ts_log, order=(2, 1, 0))  
results_AR = model.fit(disp=-1)  
plt.plot(ts_log_diff)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_AR.fittedvalues-ts_log_diff)**2))

In [ ]:
predictions_ARIMA_diff = pd.Series(results_AR.fittedvalues, copy=True)
print(predictions_ARIMA_diff.head())

In [ ]:
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
predictions_ARIMA_diff_cumsum.head()

In [ ]:
predictions_ARIMA_log = pd.Series(ts_log.ix[0], index=ts_log.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA_log.head()

In [ ]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)

In [ ]:
predictions_ARIMA=predictions_ARIMA.groupby('Start_Time_MM_DD_YYYY').head(6)

### 4.3 Prepare the Dataset for Dashboard

In [ ]:
data=data.groupby('outgoing_site_id').head(6)
data

In [ ]:
merged_data=pd.merge(data,pd.DataFrame({"Call Drop Predictions":predictions_ARIMA.astype(int)}), how='inner', left_index=True, right_index=True)
merged_data

In [ ]:
merged_data['Predicted_Call_Drop_Perc']=round((merged_data['Call Drop Predictions']/merged_data['Total Calls'])*100,2)
merged_data

In [ ]:
merged_data=merged_data.groupby(['outgoing_site_id','Start_Time_HH_MM_SS_s']).head(1)
merged_data

In [ ]:
merged_data=merged_data.reset_index()
del merged_data['Start_Time_MM_DD_YYYY']

In [ ]:
merged_data = merged_data.replace(np.nan, 0, regex=True)

In [ ]:
merged_data

## 4.4 Store Data back to the Db

In [ ]:
dsn_driver = "IBM DB2 ODBC DRIVER"
dsn_database = credentials_1['database'] 
dsn_hostname = credentials_1['host']
dsn_port = 50000               
dsn_uid = credentials_1['username']      
dsn_pwd = credentials_1['password']

dsn = (
    "DRIVER={{IBM DB2 ODBC DRIVER}};"
    "DATABASE="+str(dsn_database)+";"
    "HOSTNAME="+str(dsn_hostname)+";"
    "PORT="+str(dsn_port)+";"
    "PROTOCOL=TCPIP;"
    "UID="+str(dsn_uid)+";"
    "PWD="+str(dsn_pwd)+";").format(dsn_database, dsn_hostname, dsn_port, dsn_uid, dsn_pwd)



conn = ibm_db.connect(dsn, "", "")

### Note: Replace <Schema_Name> with an existing Schema name in your db

In [ ]:
# Replace the Schema Name with the actual name
create_statement=pd.io.sql.get_schema(merged_data.reset_index(), 'TIMESERIES_DATA')
create_statement=create_statement.replace('TEXT', 'VARCHAR(500)')
create_statement=create_statement.replace('TIMESTAMP', 'VARCHAR(500)')
#create_statement= create_statement + "IN "+dsn_database
print(create_statement)
ibm_db.exec_immediate(conn, create_statement)

In [ ]:
tuple_of_tuples = tuple([tuple(x) for x in merged_data.values])
i=1
for x in merged_data.values:
    vals= (i,) + tuple(x)
    print(vals)
    sql = "INSERT INTO TIMESERIES_DATA VALUES"+ str(vals)
    i=i+1
    ins_sql=ibm_db.prepare(conn, sql)
    ibm_db.execute(ins_sql)